In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Marketing Campaign.csv to Marketing Campaign.csv


In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [83]:
# Read the dataset using the correct delimiter
df = pd.read_csv('Marketing Campaign.csv', delimiter='\t')

# Display the first 5 rows to confirm the dataset is loaded correctly
print(df.head())

# Check the structure of the dataset
print(df.info())



     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3  10-02-2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
0             0

In [84]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

# Calculate percentage of missing values
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values per column:\n", missing_percentage)

# Handle missing values in `Income` (numerical column)
if missing_percentage['Income'] > 50:
    # If >50%, drop the column (not applicable here since it's only ~1%)
    df = df.drop('Income', axis=1)
else:
    # Fill missing values for `Income` with median (due to possible skewness)
    df['Income'] = df['Income'].fillna(df['Income'].median())

# Validate missing value handling
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

Percentage of missing values per column:
 ID                     0.000000
Year_Birth             0.000000
Education              0.000000
Marital_Status         0.000000
Income                 1.071429
Ki

In [85]:
# Check for missing values
print(df.isnull().sum())  # Count missing values in each column
missing_percentage = df.isnull().mean() * 100  # Calculate the percentage of missing values
print(missing_percentage)


ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64
ID                     0.0
Year_Birth             0.0
Education              0.0
Marital_Status         0.0
Income                 0.0
Kidhome                0.0
Teenhome               0.0
Dt_Customer            0.0
Recency                0.0
MntWines           

In [86]:
# Drop columns with more than 50% missing values
df = df.dropna(thresh=len(df)*0.5, axis=1)


# Fill missing values in categorical columns with the mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])  # Don't use inplace=True


# Fill missing values in numerical columns with the median
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())  # Don't use inplace=True


# Detect and remove outliers using IQR
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Filter out outliers
df = df[~((df[numerical_cols] < (Q1 - 1.5 * IQR)) | (df[numerical_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]


# Check missing values after handling
print(df.isnull().sum())  # Check again for missing values in each column


ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64


In [87]:
import pandas as pd
# Load the dataset with tab separation (assuming it's tab-separated)
df = pd.read_csv('Marketing Campaign.csv', sep='\t')

# Strip extra spaces from column names, if any
df.columns = df.columns.str.strip()

# Check available columns in the dataset
print("Columns in the dataset:")
print(df.columns)

# Ensure the column names are correct
categorical_cols = ['Education', 'Marital_Status', 'Complain']
ordinal_cols = ['Education', 'Marital_Status']

# Check if these columns are actually present in the dataset
for col in categorical_cols + ordinal_cols:
    if col not in df.columns:
        print(f"Warning: Column '{col}' is missing from the dataset.")
    else:
        print(f"Column '{col}' is present.")

# One-Hot Encoding for categorical columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Label Encoding for ordinal columns (using LabelEncoder from sklearn)
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()

# Apply Label Encoding to ordinal columns
for col in ordinal_cols:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])
    else:
        print(f"Warning: Column '{col}' not found in the dataset for Label Encoding.")

# Display the updated DataFrame to verify the changes
print(df.head())


Columns in the dataset:
Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')
Column 'Education' is present.
Column 'Marital_Status' is present.
Column 'Complain' is present.
Column 'Education' is present.
Column 'Marital_Status' is present.
     ID  Year_Birth   Income  Kidhome  Teenhome Dt_Customer  Recency  \
0  5524        1957  58138.0        0         0  04-09-2012       58   
1  2174        1954  46344.0        1         1  08-03-2014       38   
2  4141        1965  71613.0        0         0  21-08-2013       26   
3  6

In [88]:
# Define features (X) and target (y)
X = df.drop('Response', axis=1)  # Features (independent variables)
y = df['Response']  # Target (dependent variable)

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Load the dataset with tab separation (assuming it's tab-separated)
df = pd.read_csv('Marketing Campaign.csv', sep='\t')

# Strip extra spaces from column names, if any
df.columns = df.columns.str.strip()

# Check available columns in the dataset
print("Columns in the dataset:")
print(df.columns)

# Convert 'Dt_Customer' to datetime and extract year, month, and day if it exists
if 'Dt_Customer' in df.columns:
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
    df['Customer_Year'] = df['Dt_Customer'].dt.year
    df['Customer_Month'] = df['Dt_Customer'].dt.month
    df['Customer_Day'] = df['Dt_Customer'].dt.day
    df = df.drop(columns=['Dt_Customer'])  # Drop the original date column

# Define categorical columns that need encoding
categorical_cols = ['Education', 'Marital_Status', 'Complain']

# One-Hot Encoding for categorical columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Separate the features (X) and target variable (y)
X = df.drop(columns=['Response'])  # Assuming 'Response' is the target variable
y = df['Response']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the Random Forest model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Columns in the dataset:
Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')
Accuracy: 0.875
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       577
           1       0.63      0.27      0.38        95

    accuracy                           0.88       672
   macro avg       0.76      0.62      0.66       672
weighted avg       0.85      0.88      0.85       672

[[562  15]
 [ 69  26]]
